# HK Variance Test

### This notebook provides a test for the noise and stability of the housekeeping data captured by the FPE.  It evaluates the performance of the housekeeping system by taking a large number of samples, collecting the results for each signal channel, and evaluating the variance of the data for each channel.

### Instructions:
Enter Your Name and today's date:

Enter the part numbers and serial numbers of the units under test:

When the test is complete, save this notebook as a new file, indicating the date, as "HK_Variance_Results_YYMMDD.ipynb".  Alternatively, export the notebook as a PDF file, then clear all entries and outputs.

### Start the Observatory Simulator and Load the FPE FPGA

Remember that whenever you power-cycle the Observatory Simulator, you should set `preload=True` below.

When you are running this notebook and it has *not* been power cycled, you should set `preload=False`.

Run the following cell to get the FPE loaded:

In [1]:
from tessfpe.dhu.fpe import FPE
from tessfpe.dhu.unit_tests import check_house_keeping_voltages
fpe1 = FPE(1, debug=False, preload=True, FPE_Wrapper_version='6.1.1')
print fpe1.version
fpe1.cmd_start_frames()
fpe1.cmd_stop_frames()
if check_house_keeping_voltages(fpe1):
    print "Wrapper load complete. Interface voltages OK."

Observatory Simulator Version 1.6c - Oct  8 2015 11:55:03
Wrapper load complete. Interface voltages OK.


### Set all the operating parameters to the default values:

In [2]:
def set_fpe_defaults(fpe):
    "Set the FPE to the default operating parameters and return a list of the default values"
    defaults = {}
    for k in range(len(fpe.ops.address)):
        if fpe.ops.address[k] is None:
            continue
        fpe.ops.address[k].value = fpe.ops.address[k].default
        defaults[fpe.ops.address[k].name] = fpe.ops.address[k].default
    return defaults
# Un-comment the following to see the default values:
# set_fpe_defaults(fpe1)

### Run the variance test:

In [4]:
from numpy import var
samples=100
from tessfpe.data.housekeeping_channels import housekeeping_channels
# We make sample_data a dictionary and each value will be a set of HK 
# data, with key = sample_name.
sample_data = {}

# For later:
signal_names = []
signal_values = []
signal_data = {}
variance_values = {}
    
#my_dict["new key"] = "New value"

for i in range(samples):
    # Get a new set of HK values
    house_keeping_values = fpe1.house_keeping["analogue"]
    data_values = house_keeping_values.values()
    # Add the new HK values to the sample_data dictionary:
    sample_number = "sample_" + str(i)
    sample_data[sample_number] = data_values

# Get the signal names for use later
signal_names = house_keeping_values.keys()

"""Assign the set of all HK values of the same signal (e.g. substrate_1) 
to the dictionary 'signal_data'"""

for k in range(len(signal_names)):
    # Build the list 'signal_values' for this signal:
    for i in range(samples):
        sample_number = "sample_" + str(i)
        signal_values.append(sample_data[sample_number][k])
    # Add signal_values to the signal_data dictionary:
    signal_data[signal_names[k]] = signal_values
    signal_values = []

""" Now get the variance of each of the 'signal_values' in the 
signal_data dictionary and put the result in the 'variance_values' 
dictionary."""
for name in signal_data:
    variance_values[name] = var(signal_data[name])
    # print name, str(variance_values[name])
    print '{0}      {1:<5}'.format(name, variance_values[name])

# Results will be displayed below, in engineering units.  
# Watch ObsSim LEDs for activity.

ccd4_substrate      2.66638766974e-05
ccd2_output_drain_a      1.28402558248e-06
ccd2_output_source_a      7.27075408212e-07
ccd2_output_source_c      6.39895769767e-07
ccd2_output_source_b      8.20363174192e-07
ccd1_parallel_high      3.65014281124e-07
heater_1_current      0.000105025088321
ccd4_parallel_low      8.19887593389e-07
ccd1_output_source_d      6.37119348161e-07
ccd3_output_source_a      9.0865338128e-07
ccd3_output_source_b      7.01046455651e-07
ccd3_output_source_c      7.50397349707e-07
ccd3_output_source_d      6.99311192147e-07
ccd4_output_source_d      8.10368056409e-07
ccd4_output_source_c      7.14303868822e-07
ccd4_output_source_b      8.39867535978e-07
ccd4_output_source_a      6.29900651983e-07
ccd3_board_temperature      0.000312985628843
ccd2_input_gate_2      6.23004022054e-07
ccd4_serial_low      5.62354247086e-07
ccd2_output_gate      5.36796147935e-07
ccd4_input_diode_low      4.49776905589e-07
ccd3_substrate      5.87464462966e-05
ccd2_serial_low      

## End HK Variance Test.